In [154]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [100]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D, AveragePooling2D

In [191]:
curr_path = os.getcwd()
input_path = os.path.join(curr_path,'merged_data_1020.xlsx')

data = pd.read_excel(
    input_path,
    engine='openpyxl',
)
data.dropna(inplace = True)

In [193]:
withouty = data.iloc[:, 2:]
withouty
normalizeddata = (withouty - withouty.mean()) / withouty.std()

In [194]:
normalizeddata['SP500-EPS-Index'] = data['SP500-EPS-Index']

In [195]:
# a helper function concats data among lagged window (length=lags) with the target
# e.g, for input dat with var = [date, X, y], lags = 2, 
#  the returned dataset will have colunn names [date_2, X_2, y_2, date_1, X_1, y_1, date, X, y] with length - 2

def flatten_data(dat, lags = 3):
    new_data = dat.copy()

    # add variables you don't want to include below
    dropped_col = ['gdp']
    for i in range(dat.shape[1]):
        new_data.columns.values[i] = new_data.columns.values[i] + '_'  + str(0)
    
    #new_data.drop(dropped_col, axis=1, inplace=True)
    
    for n in range(lags):
        fill_data = dat.shift(periods = n+1)
        fill_data.dropna(inplace=True)     
        new_data = new_data.iloc[1:, :]
        new_data = pd.concat([fill_data, new_data], axis=1)
        
        # rename added columns
        for i in range(dat.shape[1]):
            new_data.columns.values[i] = dat.columns.values[i][:-2] + '_'  + str(n+1)
        new_dropped_col=[]
        for i in range(len(dropped_col)):
            name = dropped_col[i]+'_'+str(n+1)
            new_dropped_col.append(name)
        new_data.drop(new_dropped_col, axis=1, inplace=True)
            
    return new_data

In [196]:
flatterned = flatten_data(normalizeddata, lags = 3)

In [197]:
flatterned.drop(['gdp_0'], axis = 1, inplace = True)

In [198]:
flatterned.shape

(319, 60)

In [199]:
# train ratio = 0.8
def train_test_split(data, ratio):
    n = len(data)
    train_idx = round(n*ratio)
    test_idx = round(n*ratio) + 1
    train = data[:train_idx]
    test = data[test_idx:]
    # find train_y, train_x and test_y, test_x
    train_y = train['SP500-EPS-Index_0']
    train_x = train.loc[:, data.columns[:45]]
    test_y = test['SP500-EPS-Index_0']
    test_x = test.loc[:, data.columns[:45]]
    return train_y, test_y, train_x, test_x

In [205]:
train_y, test_y, train_x, test_x = train_test_split(flatterned, 0.95)

In [206]:
# reshape data and normalize data
train_x = np.array(train_x).reshape(len(train_x), 3, 15, 1)
test_x = np.array(test_x).reshape(len(test_x), 3, 15, 1)

In [207]:
# build CNN model
model = Sequential()
model.add(Conv2D(filters = 32, 
                 input_shape = ((3,15,1)),
                 data_format = 'channels_last',
                 kernel_size=(2,2), 
                 strides=(1,1),   
                 activation='relu'))
#model.add(MaxPooling2D(pool_size=(2, 1)))
#model.add(AveragePooling2D(pool_size=(2,1)))
model.add(Flatten())
model.add(Dense(45, activation='relu'))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

Model: "sequential_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_38 (Conv2D)           (None, 2, 14, 32)         160       
_________________________________________________________________
flatten_11 (Flatten)         (None, 896)               0         
_________________________________________________________________
dense_30 (Dense)             (None, 45)                40365     
_________________________________________________________________
dense_31 (Dense)             (None, 1)                 46        
Total params: 40,571
Trainable params: 40,571
Non-trainable params: 0
_________________________________________________________________


In [208]:
model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=50)

Epoch 1/50
10/10 [==============================] - 0s 17ms/step - loss: 1133732.7841 - val_loss: 3679912.0000
Epoch 2/50
10/10 [==============================] - 0s 8ms/step - loss: 806421.5398 - val_loss: 2584797.2500
Epoch 3/50
10/10 [==============================] - 0s 15ms/step - loss: 609464.4886 - val_loss: 1416420.7500
Epoch 4/50
10/10 [==============================] - 0s 14ms/step - loss: 277419.8111 - val_loss: 426413.0000
Epoch 5/50
10/10 [==============================] - 0s 8ms/step - loss: 72301.9822 - val_loss: 10664.6992
Epoch 6/50
10/10 [==============================] - 0s 7ms/step - loss: 6830.3675 - val_loss: 36443.7656
Epoch 7/50
10/10 [==============================] - 0s 7ms/step - loss: 14106.1883 - val_loss: 17648.1992
Epoch 8/50
10/10 [==============================] - 0s 7ms/step - loss: 6118.6812 - val_loss: 1181.9240
Epoch 9/50
10/10 [==============================] - 0s 9ms/step - loss: 4418.1312 - val_loss: 2269.4319
Epoch 10/50
10/10 [=================

In [209]:
data

,Date,SP500-Price,SP500-EPS-Index_0,gdp_0,IndustrialProduction_0,GS10_0,ManufacturingNewOrders_0,CPIFABSL_0,MedianUsualWeeklyRealEarnings_0,PCE_0,PPI_0,ConstructionSpending_0,VolumeOfTotalRetailTradeSales-GR_0,HousingSales_0,UnemploymentRate_0,EffectiveRate_0,M2_0,GovernmentBudgetBalance_0
281,1993-01-01,435.23,329.444289,6729.459,61.9438,6.60,239924.0,140.000,315.0,4340.7,118.0,458080.0,66.295964,118000,7.3,3.02,3423.6,-63.127
282,1993-02-01,441.70,333.600376,6808.939,62.0967,6.26,247176.0,140.400,317.0,4355.3,118.4,462967.0,65.639146,129400,7.1,3.03,3400.7,-17.571
283,1993-03-01,450.16,337.892136,6808.939,61.9756,5.98,245506.0,140.400,317.0,4352.5,118.7,458399.0,64.910899,125000,7.0,3.07,3410.5,-17.571
284,1993-04-01,443.08,334.939705,6808.939,62.3009,5.97,244703.0,140.800,317.0,4393.4,119.3,469425.0,66.449715,127000,7.1,2.96,3430.4,-17.571
285,1993-05-01,445.25,332.157218,6882.098,62.2327,6.04,245194.0,141.700,319.0,4422.4,119.7,468998.0,66.943896,129900,7.1,3.00,3425.0,-53.582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,2019-06-01,2890.17,2303.374446,21505.012,99.6115,2.07,480142.0,257.707,360.0,14425.7,200.3,1385267.0,106.099185,311800,3.6,2.38,14753.8,-237.915
599,2019-07-01,2996.11,2290.278233,21505.012,99.2139,2.06,486699.0,257.998,360.0,14487.4,200.7,1409115.0,106.615519,308300,3.6,2.40,14833.1,-237.915
600,2019-08-01,2897.50,2277.247380,21694.458,99.7598,1.63,481697.0,258.158,362.0,14536.4,199.2,1419321.0,107.118994,327000,3.7,2.13,14914.7,-356.353
601,2019-09-01,2982.16,2263.360675,21694.458,99.1341,1.70,481257.0,258.740,362.0,14564.7,198.4,1427866.0,106.090787,315700,3.5,2.04,15005.6,-356.353
